In [ ]:
import pandas as pd
import requests
import zipfile
import pandas as pd
import os
from io import BytesIO

Extract data from url, unzip and charge to a dataframe

In [6]:
# URL of the ZIP file
url = 'https://archive.ics.uci.edu/static/public/20/census+income.zip'

# Download the ZIP file
response = requests.get(url)
zip_content = BytesIO(response.content)

# Extract the ZIP file
with zipfile.ZipFile(zip_content, 'r') as zip_ref:
    zip_ref.extractall("data_census")

# Path to the CSV file within the extracted directory
csv_file = 'data_census/adult.data'

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file, header=None)

# Display the first 5 rows of the DataFrame
df.sample(5)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
20806,54,Self-emp-inc,129432,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K
31088,23,Private,133582,1st-4th,2,Never-married,Farming-fishing,Not-in-family,White,Male,0,0,36,Mexico,<=50K
21736,29,Private,184078,HS-grad,9,Never-married,Other-service,Other-relative,White,Female,0,0,40,United-States,<=50K
8172,22,Private,48347,Bachelors,13,Never-married,Farming-fishing,Own-child,White,Male,0,0,40,United-States,<=50K
12556,40,Private,114537,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,>50K
